In [1]:
import pandas as pd 
import numpy as np 


In [2]:
map_df = pd.read_csv(r'D:\python\financial statement prj\group_2_chatbot_financial_statement\csv\map_category_code_sec.csv')
data_df = pd.read_csv(r'D:\python\financial statement prj\group_2_chatbot_financial_statement\csv\securities_financial_report_v2_1.csv')


In [3]:
map_df.head()

,category_code,en_caption,vi_caption,report_type
0,BS_100,A. CURRENT ASSETS,A. TÀI SẢN NGẮN HẠN,balance_sheet
1,BS_110,I. Financial Assets,I. Tài sản tài chính,balance_sheet
2,BS_111,1. Cash and cash equivalents,1. Tiền và các khoản tương đương tiền,balance_sheet
3,BS_111.1,1.1. Cash,1.1. Tiền,balance_sheet
4,BS_111.2,1.2. Cash equivalents,1.2. Các khoản tương đương tiền,balance_sheet


In [4]:
data_df.head()

,stock_code,category_code,year,quarter,data
0,MBS,IS_001,2020,0,202471
1,MBS,IS_001,2021,0,370485
2,MBS,IS_001,2022,0,226222
3,MBS,IS_001,2023,0,0
4,MBS,IS_001,2016,0,42664


## Phân tích cấu trúc tài chính 

In [5]:
import pandas as pd

# Pivot table to restructure financial data based on category codes
pivot_df = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                           columns='category_code', 
                                           values='data', 
                                           aggfunc='sum')

# Ratio calculation functions
def equity_ratio(equity, total_assets):
    return equity / total_assets if total_assets else None

def long_term_asset_self_financing_ratio(permanent_capital, long_term_assets):
    return permanent_capital / long_term_assets if long_term_assets else None

def fixed_asset_self_financing_ratio(permanent_capital, fixed_assets):
    return permanent_capital / fixed_assets if fixed_assets else None

def general_solvency_ratio(total_assets, total_liabilities):
    return total_assets / total_liabilities if total_liabilities else None

def return_on_investment(net_income, total_investment):
    return net_income / total_investment if total_investment else None

def ROIC(EBIT,Tax_expense, invested_capital):
    return (EBIT*(1 - (Tax_expense/EBIT))) / invested_capital if invested_capital else None

def return_on_long_term_capital(EBIT, avg_long_term_capital):
    return EBIT / avg_long_term_capital if avg_long_term_capital else None

def basic_earning_power(EBIT, average_total_assets):
    return EBIT / average_total_assets if average_total_assets else None

def debt_to_assets_ratio(total_liabilities, total_assets):
    return total_liabilities / total_assets if total_assets else None

def debt_to_equity_ratio(total_liabilities, equity):
    return total_liabilities / equity if equity else None

def short_term_debt_to_assets_ratio(short_term_liabilities, total_assets):
    return short_term_liabilities / total_assets if total_assets else None

def interest_coverage_ratio(EBIT, interest_expense):
    return EBIT / interest_expense if interest_expense else None

def long_term_debt_to_equity_ratio(long_term_liabilities, equity):
    return long_term_liabilities / equity if equity else None

def short_term_debt_to_equity_ratio(short_term_liabilities, equity):
    return short_term_liabilities / equity if equity else None

# Dictionary to hold functions and corresponding category codes
ratio_functions = {
    'equity_ratio': ['BS_400', 'BS_270'],  # equity, total_assets
    'long_term_asset_self_financing_ratio': [['BS_410','BS_340'], 'BS_200'],  # permanent_capital (equity+long-term liabilities ), long_term_assets (non-current assets)
    'fixed_asset_self_financing_ratio': [['BS_410','BS_340'], 'BS_220'],  # permanent_capital (equity ), fixed_assets
    'general_solvency_ratio': ['BS_270', 'BS_300'],  #  total_assets, total_liabilities
    'return_on_investment': ['IS_200', 'BS_212'],  # net_income, total_investment (using total assets as example)
    'ROIC': [['IS_090','IS_052'], 'IS_100',['BS_410','BS_312','BS_346']],  # EBIT,Tax_expense, invested_capital
    'return_on_long_term_capital': [['IS_090','IS_052'], ['BS_410','BS_340']],  # EBIT, avg_long_term_capital
    'basic_earning_power': [['IS_090','IS_052'], 'BS_270'],  # EBIT, avg_total_assets
    'debt_to_assets_ratio': ['BS_300', 'BS_270'],  # total_liabilities, total_assets
    'debt_to_equity_ratio': ['BS_300', 'BS_410'],  # total_liabilities, equity
    'short_term_debt_to_assets_ratio': ['BS_310', 'BS_270'],  # short_term_liabilities, total_assets
    'interest_coverage_ratio': [['IS_090','IS_052'], 'IS_052'],  # EBIT, interest_expense
    'long_term_debt_to_equity_ratio': ['BS_340', 'BS_410'],  # long_term_liabilities(BS_350 + BS_360 + BS_372), equity
    'short_term_debt_to_equity_ratio': ['BS_310', 'BS_410']  # short_term_liabilities, equity
}

# Create a DataFrame to store the results
results_1 = []

# Iterate through pivoted data
for index, row in pivot_df.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in ratio_functions.items():
        # Fetch input data from the pivot table
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_1.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert results to DataFrame
ratios_df_1 = pd.DataFrame(results_1)

ratios_df_1


,stock_code,year,quarter,ratio_code,data
0,MBS,2016,0,equity_ratio,0.376350
1,MBS,2016,0,long_term_asset_self_financing_ratio,8.432240
2,MBS,2016,0,fixed_asset_self_financing_ratio,132.743137
3,MBS,2016,0,general_solvency_ratio,1.603464
4,MBS,2016,0,return_on_investment,0.164881
...,...,...,...,...,...
2571,VND,2024,3,debt_to_equity_ratio,NaN
2572,VND,2024,3,short_term_debt_to_assets_ratio,NaN
2573,VND,2024,3,interest_coverage_ratio,NaN
2574,VND,2024,3,long_term_debt_to_equity_ratio,NaN


In [ ]:
# # Financing Ratio (Hệ số tài trợ)
# def equity_ratio(equity, total_assets):
#     return equity / total_assets

# #  Long-Term Asset Self-Financing Ratio (Hệ số tự tài trợ tài sản dài hạn)
# # longterm_assets = BS_200
# def long_term_asset_self_financing_ratio(permanent_capital, long_term_assets):
#     return permanent_capital / long_term_assets

# #Fixed Asset Self-Financing Ratio (Hệ số tự tài trợ tài sản cố định - TSCĐ)
# # permanent capital = VCSH(400)+ Nợ dài hạn (330)
# # fixed assets = TSCĐ đã và đang đầu tư(220 + 240/242)
# def fixed_asset_self_financing_ratio(permanent_capital, fixed_assets):
#     return permanent_capital / fixed_assets

# # General Solvency Ratio (Hệ số khả năng thanh toán tổng quát) 
# def general_solvency_ratio(total_liabilities, total_assets):
#     return total_assets / total_liabilities

# # ROI 
# def return_on_investment(net_profit, total_investment):
#     return net_profit / total_investment

# # ROIC (Return on Invested Capital)
# #Net Operating Profit After Taxes (NOPAT) = 60
# def ROIC (NOPAT, invested_capital):
#     return NOPAT / invested_capital

# # Return on Long-Term Capital
# # Earnings Before Interest and Taxes (EBIT) (Lợi nhuận trước thuế và lãi vay) = 50
# def return_on_long_term_capital(EBIT, average_long_term_capital):
#     return EBIT / average_long_term_capital

# # Basic Earning Power (BEP) (Năng lực sinh lời cơ bản)
# def basic_earning_power(EBIT, average_total_assets):
#     return EBIT / average_total_assets

# #Debt-to-Assets Ratio
# def debt_to_assets_ratio(total_liabilities, total_assets):
#     return total_liabilities / total_assets

# # Debt-to-Equity Ratio
# def debt_to_equity_ratio(total_liabilities, equity):
#     return total_liabilities / equity

# # Short-Term Debt-to-Assets Ratio
# def short_term_debt_to_assets_ratio(short_term_liabilities, total_assets):
#     return short_time_liabilities / total_assets

# #Equity-to-Total Assets Ratio:
# def equity_to_total_assets_ratio(equity, total_assets):
#     return equity / total_assets

# #Interest Coverage Ratio:
# def interest_coverage_ratio(EBIT, interest_expense):
#     return EBIT / interest_expense
# # Long-Term Debt-to-Equity Ratio
# #longterm liabilities = BS_330
# def long_term_debt_to_equity_ratio(long_term_liabilities, equity):
#     return long_term_liabilities / equity

# # Short-Term Debt-to-Total Assets Ratio:
# def short_term_debt_to_total_assets_ratio(short_term_liabilities, total_assets):
#     return short_term_liabilities / total_assets



## Phân tích khả năng thanh toán (LIquidity)

In [ ]:
# # Receivables-to-Payables Ratio (Tỷ lệ giữa nợ phải thu và nợ phải trả)
# # accounts_receivable(130+210) / total_liabilities(310+330)
# def receivables_to_payables_ratio(accounts_receivable, total_liabilities):
#     return accounts_receivable / total_liabilities

# #Receivables-to-Total Assets Ratio
# def receivables_to_total_assets_ratio(accounts_receivable, total_assets):
#     return accounts_receivable / total_assets

# #Debt-to-Total Capital Ratio 
# # total_liabilities = BS_300
# # total_capital = BS_440
# def debt_to_total_capital_ratio(total_liabilities, total_capital):
#     return total_liabilities / total_capital

# # the Receivables-to-Sales Ratio (Tỷ lệ giữa nợ phải thu người mua so với tổng số tiền hàng bán ra trong kỳ) 
# # accouts_receivables(131+211) / total_sales (10)
# def receivables_to_sales_ratio(accounts_receivables, total_sales):
#     return accounts_receivables / total_sales

# # Allowance for Doubtful Accounts Ratio (Tỷ lệ giữa dự phòng nợ phải thu khó đòi so với tổng số nợ phải thu người mua)
# # allowance_for_doubtful_accounts(137+219) / accounts_receivables(131+211)
# def allowance_for_doubtful_accounts_ratio(allowance_for_doubtful_accounts, accounts_receivables):
#     return allowance_for_doubtful_accounts / accounts_receivables

# #Asset-to-Debt Ratio
# def asset_to_debt_ratio(total_assets, total_liabilities):
#     return total_assets / total_liabilities

# # Current ratio (Hệ số thanh khoản ngắn hạn)
# # current_assets = BS_100 - BS_151
# def current_ratio(current_assets, current_liabilities):
#     return current_assets / current_liabilities

# # quick ratio (Hệ số thanh khoản nhanh)
# # current assets - inventory (BS_100 - BS_151 - BS_140) / current liabilities
# def quick_ratio(current_assets, inventory, current_liabilities):
#     return (current_assets - inventory) / current_liabilities

# # Cash Ratio (Hệ số thanh khoản tiền mặt)
# # cash_and_cash_equivalents(BS_110) / current_liabilities
# def cash_ratio(cash_and_cash_equivalents, current_liabilities):
#     return cash_and_cash_equivalents / current_liabilities

# # Current Maturity Coverage Ratio= Cash and Cash Equivalents/ Current Maturing Debt 
# # cash_and_cash_equivalents(BS_110) / current_maturing_debt ( TMBCTC)

# # long_term_debt_coverage_ratio = non_current_assets / long_term_debt
# def long_term_debt_coverage_ratio(non_current_assets, non_current_liabilities):
#     return non_current_assets / non_current_liabilities
# # Debt-to-Equity Ratio = Total Liabilities/ Total Equity
# def debt_to_equity_ratio(total_liabilities, total_equity):
#     return total_liabilities / total_equity

# # Long-Term Debt to Equity Capital Ratio
# # non_current_liabilities = BS_330
# def long_term_debt_to_equity_capital_ratio(non_current_liabilities, equity):
#     return non_current_liabilities / equity

# # Time Interest Earned (TIE) = EBIT(IS_050 + IS_023)/ Interest Expense(IS_023)
# def time_interest_earned(EBIT, interest_expense):
#     return EBIT / interest_expense

# # Debt to Tangible Net Worth Ratio = total_liabilities / equity - intangible_assets (BS_227)
# def debt_to_tangible_net_worth_ratio(total_liabilities, equity, intangible_assets):
#     return total_liabilities / (equity - intangible_assets)



 


In [7]:
import pandas as pd

# Pivot table to restructure financial data based on category codes
pivot_df_2 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                           columns='category_code', 
                                           values='data', 
                                           aggfunc='sum')

# Ratio calculation functions
def receivables_to_payables_ratio(accounts_receivable, total_liabilities):
    return accounts_receivable / total_liabilities if total_liabilities else None

def receivables_to_total_assets_ratio(accounts_receivable, total_assets):
    return accounts_receivable / total_assets if total_assets else None

def debt_to_total_capital_ratio(total_liabilities, total_capital):
    return total_liabilities / total_capital if total_capital else None

def receivables_to_sales_ratio(accounts_receivables, total_sales):
    return accounts_receivables / total_sales if total_sales else None

# def allowance_for_loan_customers_ratio(allowance_for_loan_customers, loan_to_customers):
#     return allowance_for_loan_customers / loan_to_customers if loan_to_customers else None

def asset_to_debt_ratio(total_assets, total_liabilities):
    return total_assets / total_liabilities if total_liabilities else None

def current_ratio(current_assets, current_liabilities):
    return current_assets / current_liabilities if current_liabilities else None

def quick_ratio(current_assets, inventory, current_liabilities):
    return (current_assets - inventory) / current_liabilities if current_liabilities else None

def cash_ratio(cash_and_cash_equivalents, current_liabilities):
    return cash_and_cash_equivalents / current_liabilities if current_liabilities else None

def long_term_debt_coverage_ratio(non_current_assets, non_current_liabilities):
    return non_current_assets / non_current_liabilities if non_current_liabilities else None

def debt_to_equity_ratio(total_liabilities, total_equity):
    return total_liabilities / total_equity if total_equity else None

def long_term_debt_to_equity_capital_ratio(non_current_liabilities, equity):
    return non_current_liabilities / equity if equity else None
#ICR
def time_interest_earned(EBIT, interest_expense):
    return EBIT / interest_expense if interest_expense else None

def debt_to_tangible_net_worth_ratio(total_liabilities, equity, intangible_assets):
    return total_liabilities / (equity - intangible_assets) if (equity - intangible_assets) else None

ratio_functions = {
    'receivables_to_payables_ratio': [ 'BS_117', 'BS_300'],  # accounts_receivable, total_liabilities
    'receivables_to_total_assets_ratio': [ 'BS_117', 'BS_270'],  # accounts_receivable, total_assets
    'debt_to_total_capital_ratio': ['BS_300', ['BS_411']],  # total_liabilities, total_capital
    'receivables_to_sales_ratio': ['BS_117', ['IS_010','IS_020','IS_050','IS_006','IS_007','IS_008','IS_009']],  # accounts_receivables, total_sales
    'asset_to_debt_ratio': ['BS_270', 'BS_300'],  # total_assets, total_liabilities
    'current_ratio': ['BS_100','BS_310'],  # current_assets_for_liquidity(), current_liabilities
    # 'quick_ratio': [['BS_100', 'BS_151'], 'BS_140', 'BS_310'],  # current_assets - inventory, current_liabilities
    'cash_ratio': ['BS_111', 'BS_310'],  # cash_and_cash_equivalents, current_liabilities
    'long_term_debt_coverage_ratio': ['BS_200', 'BS_340'],  # non_current_assets, non_current_liabilities
    'debt_to_equity_ratio': ['BS_300', 'BS_400'],  # total_liabilities, total_equity
    'long_term_debt_to_equity_capital_ratio': ['BS_340', 'BS_400'],  # long_term_liabilities, equity
    'time_interest_earned': ['IS_090', 'IS_052'],  # EBIT, interest_expense
    'debt_to_tangible_net_worth_ratio': ['BS_300', 'BS_400', 'BS_227'],  # total_liabilities, equity, intangible_assets
}

# Create a DataFrame to store the results
results_2 = []

# Iterate through pivoted data
for index, row in pivot_df_2.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in ratio_functions.items():
        input_values = []
            # For all other ratios, sum values if needed or fetch directly
        for input_name in inputs:
            if isinstance(input_name, list):
                 value_sum = sum([row[i] for i in input_name if i in row.index])
                 input_values.append(value_sum)
            else:
                 input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_2.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert results to DataFrame
ratios_df_2 = pd.DataFrame(results_2)

# Display or export the DataFrame
ratios_df_2


,stock_code,year,quarter,ratio_code,data
0,MBS,2016,0,receivables_to_payables_ratio,0.009908
1,MBS,2016,0,receivables_to_total_assets_ratio,0.006179
2,MBS,2016,0,debt_to_total_capital_ratio,1.796777
3,MBS,2016,0,receivables_to_sales_ratio,0.032390
4,MBS,2016,0,asset_to_debt_ratio,1.603464
...,...,...,...,...,...
2203,VND,2024,3,long_term_debt_coverage_ratio,NaN
2204,VND,2024,3,debt_to_equity_ratio,NaN
2205,VND,2024,3,long_term_debt_to_equity_capital_ratio,NaN
2206,VND,2024,3,time_interest_earned,NaN


## Phân tích rủi ro tài chính (Financial risk)

In [ ]:
# # financial_leverage = total_liabilities(BS_300) / total_lia_and_equity (BS_440)
# # 0< financial_leverage < 1 : Close to 0 is better
# def financial_leverage(total_liabilities, total_lia_and_equity):
#     return total_liabilities / total_lia_and_equity

# # Allowance for Doubtful Accounts to Total Assets Ratio = allowance_for_doubtful_accounts(BS_137+BS_219)/ total_assets (BS_270)
# def allowance_for_doubtful_accounts_to_total_assets_ratio(allowance_for_doubtful_accounts, total_assets):
#     return allowance_for_doubtful_accounts / total_assets

# # Hệ số tài trợ thường xuyên (Permanent Financing Ratio) = permanent_capital(BS_400+BS_330) / total_lia_and_equity (BS_440)
# def permanent_financing_ratio(permanent_capital, total_lia_and_equity):
#     return permanent_capital / total_lia_and_equity





In [8]:
import pandas as pd

# Create a new pivot table for the new ratios
pivot_df_3 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                 columns='category_code', 
                                 values='data', 
                                 aggfunc='sum')


def financial_leverage(total_liabilities, total_lia_and_equity):
    return total_liabilities / total_lia_and_equity if total_lia_and_equity else None


def allowance_for_doubtful_accounts_to_total_assets_ratio(allowance_for_doubtful_accounts, total_assets):
    return allowance_for_doubtful_accounts / total_assets if total_assets else None


def permanent_financing_ratio(permanent_capital, total_lia_and_equity):
    return permanent_capital / total_lia_and_equity if total_lia_and_equity else None

new_ratio_functions = {
    'financial_leverage': ['BS_300', 'BS_440'],  # total_liabilities , total_lia_and_equity 
    'allowance_for_doubtful_accounts_to_total_assets_ratio': ['BS_129', 'BS_270'],  # allowance_for_doubtful_accounts , total_assets 
    # 'permanent_financing_ratio': [, 'BS_440'],  # permanent_capital (BS_400 + BS_330), total_lia_and_equity 
}

# Create a DataFrame to store the results of the new ratios
results_3 = []

# Iterate through the pivot table to calculate the new ratios
for index, row in pivot_df_3.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in new_ratio_functions.items():
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  # Sum for cases like permanent capital or allowance for doubtful accounts
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_3.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert the results to a DataFrame
ratios_df_3 = pd.DataFrame(results_3)

# Display or export the DataFrame
ratios_df_3


,stock_code,year,quarter,ratio_code,data
0,MBS,2016,0,financial_leverage,0.623650
1,MBS,2016,0,allowance_for_doubtful_accounts_to_total_asset...,-0.145904
2,MBS,2017,0,financial_leverage,0.699253
3,MBS,2017,0,allowance_for_doubtful_accounts_to_total_asset...,-0.103755
4,MBS,2018,0,financial_leverage,0.613101
...,...,...,...,...,...
363,VND,2024,1,allowance_for_doubtful_accounts_to_total_asset...,-0.000670
364,VND,2024,2,financial_leverage,0.617213
365,VND,2024,2,allowance_for_doubtful_accounts_to_total_asset...,-0.001346
366,VND,2024,3,financial_leverage,NaN


## phân tích kết quả kinh doanh 

In [ ]:
# # Doanh thu thuần về bán hàng và cung cấp dịch vụ ,IS_010
# # Doanh thu hoạt động tài chính,IS_021,Financial income
# # doanh thu thuần = IS_010 + IS_021

# # Financial Income to Net Revenue Ratio = Financial Income(IS_021)/ Net Revenue
# def financial_income_to_net_revenue_ratio(financial_income, net_revenue):
#     return financial_income / net_revenue



In [9]:
import pandas as pd

# Create a new pivot table for the Financial Income to Net Revenue Ratio
pivot_df_4 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                 columns='category_code', 
                                 values='data', 
                                 aggfunc='sum')

# Financial Income to Net Revenue Ratio = Financial Income(IS_021) / Net Revenue
def financial_income_to_net_revenue_ratio(financial_income, net_revenue):
    return financial_income / net_revenue if net_revenue else None

# Dictionary to hold the category codes for the new ratio
new_ratio_function = {
    'financial_income_to_net_revenue_ratio': ['IS_050','IS_020']
}

# Create a DataFrame to store the results of the new ratio
results_4 = []

# Iterate through the pivot table to calculate the new ratio
for index, row in pivot_df_4.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in new_ratio_function.items():
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list): 
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_4.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert the results to a DataFrame
ratios_df_4 = pd.DataFrame(results_4)

# Display or export the DataFrame
ratios_df_4


,stock_code,year,quarter,ratio_code,data
0,MBS,2016,0,financial_income_to_net_revenue_ratio,0.004072
1,MBS,2017,0,financial_income_to_net_revenue_ratio,0.003157
2,MBS,2018,0,financial_income_to_net_revenue_ratio,0.004058
3,MBS,2018,1,financial_income_to_net_revenue_ratio,NaN
4,MBS,2018,2,financial_income_to_net_revenue_ratio,NaN
...,...,...,...,...,...
179,VND,2023,3,financial_income_to_net_revenue_ratio,0.004239
180,VND,2023,4,financial_income_to_net_revenue_ratio,0.006222
181,VND,2024,1,financial_income_to_net_revenue_ratio,0.003971
182,VND,2024,2,financial_income_to_net_revenue_ratio,0.004159


## phân tích khả năng sinh lời ( profitability )

In [ ]:
# # ROA : Return on Assets = net_income(IS_060) / total_assets
# def return_on_assets(net_income, total_assets):
#     return net_income / total_assets

# #Return on fixed assets = net_income(IS_060) / average_fixed_assets (BS_220)
# def return_on_fixed_assets(net_income, average_fixed_assets):
#     return net_income / average_fixed_assets

# # #RNOA= Net Operating Profit After Tax (NOPAT)/Average Net Operating Assets
# # def RNOA(NOPAT, average_net_operating_assets):
# #     return NOPAT / average_net_operating_assets

# # return on long-term operating assets = net_income(IS_060) / average_long_term_operating_assets
# def return_on_long_term_operating_assets(net_income, average_long_term_operating_assets):
#     return net_income / average_long_term_operating_assets

# # SỨC SINH LỢI CƠ BẢN CỦA TÀI SẢN : BEPR (Basic Earning Power Ratio) = EBIT(IS_050 +IS_023) / total_assets
# def BEPR(EBIT, total_assets):
#     return EBIT / total_assets

# #ROE = Net Income/  Equity
# def ROE(net_income, equity):
#     return net_income / equity

# # Return on Common Equity (sức sinh lời của vốn cổ phần thường) = Net Income (IS_060) - (Preferred Dividends-CF-036)/ Average Common Equity
# def return_on_common_equity(net_income, preferred_dividends, average_common_equity):
#     return net_income - preferred_dividends / average_common_equity

# # Profitability of Cost of Goods Sold (COGS) = net income from operating(IS_030)/ COGS (IS_011)
# def profitability_of_cost_of_goods_sold(net_income_from_operating, COGS):
#     return net_income_from_operating / COGS

# # Hệ số chênh lệch giá (Price Spread Ratio): gross_profit(IS_020) / COGS(IS_011)
# def price_spread_ratio(gross_profit, COGS):
#     return gross_profit / COGS

# # Sức sinh lợi của chi phí kinh doanh (Profitability of Operating Expenses): net_income_from_operating(IS_030) / total_operating_expenses(IS_025+IS_026+IS_011)
# def profitability_of_operating_expenses(net_income_from_operating, total_operating_expenses):
#     return net_income_from_operating / total_operating_expenses

# # ROS = Net Income/ Net Sales (IS_010)
# def ROS(net_income, net_sales):
#     return net_income / net_sales

# # adjusted ROS = net_income(IS_060) - cổ tức ưu đãi/ net_sales(IS_010) 

# # Operating Profit Margin (OPM) = Net profit from operating activities (IS_030)/ Net Sales
# def operating_profit_margin(net_profit_from_operating, net_sales):
#     return net_profit_from_operating / net_sales

# # Gross Profit Margin (GPM) = Gross Profit(IS_020)/ Net Sales
# def gross_profit_margin(gross_profit, net_sales):
#     return gross_profit / net_sales

# #net operating profit margin = lợi nhuận sau thuế từ hđkd(net operating profit after tax, NOPAT)/ net sales(IS_010)
# # NOPAT = lợi nhuận trc thuế từ hđkd x (1 - thuế suất)
# def net_operating_profit_margin(NOPAT, net_sales):
#     return NOPAT / net_sales

# # price to book ratio = giá cổ phiếu / giá trị sổ sách ( market price per share/book value per share)
# # price to earnings ratio = giá cổ phiếu / lợi nhuận trên cổ phiếu
# #Dividend Payout Ratio =  Dividend per Share/ Earnings per Share
# # Dividend Yield = Dividend per Share/ Market Price per Share





In [12]:
import pandas as pd

def get_previous_year_q0_value(stock_code, year, category_code):
    try:
        return pivot_df_5.loc[(stock_code, year - 1, 0), category_code]
    except KeyError:
        return None

# Create a new pivot table for the ratios in pivot_df_5
pivot_df_5 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                 columns='category_code', 
                                 values='data', 
                                 aggfunc='sum')

# Ratio calculation functions
def return_on_assets(net_income, total_assets):
    return net_income / total_assets if total_assets else None

def return_on_fixed_assets(net_income, average_fixed_assets):
    return net_income / average_fixed_assets if average_fixed_assets else None

def return_on_long_term_operating_assets(net_income, average_long_term_operating_assets):
    return net_income / average_long_term_operating_assets if average_long_term_operating_assets else None

def Basic_Earning_Power_Ratio(EBIT, total_assets):
    return EBIT / total_assets if total_assets else None

def Return_on_equity(net_income, equity):
    return net_income / equity if equity else None

# def return_on_common_equity(net_income, preferred_dividends, average_common_equity):
#     return (net_income - preferred_dividends) / average_common_equity if average_common_equity else None

def profitability_of_cost_of_goods_sold(net_income_from_operating, COGS):
    return net_income_from_operating / COGS if COGS else None

def price_spread_ratio(gross_profit, COGS):
    return gross_profit / COGS if COGS else None

def profitability_of_operating_expenses(profit_from_operating, operating_expense, total_operating_expenses):
    return  (profit_from_operating - operating_expense) / total_operating_expenses if total_operating_expenses else None

def Return_on_sales(net_income, net_sales):
    return net_income / net_sales if net_sales else None

def operating_profit_margin(profit_from_operating,operating_expense, net_sales):
    return (profit_from_operating - operating_expense)/ net_sales if net_sales else None

def gross_profit_margin(gross_profit, net_sales):
    return gross_profit / net_sales if net_sales else None

# Dictionary to hold functions and corresponding category codes
ratio_functions_5 = {
    'return_on_assets': ['IS_200', 'BS_270'],  # net_income , total_assets 
    'return_on_fixed_assets': ['IS_200', 'BS_220'],  # net_income , average_fixed_assets 
    'return_on_long_term_operating_assets': ['IS_200', 'BS_200'],  # net_income , average_long_term_operating_assets 
    'Basic_Earning_Power_Ratio': ['IS_090', 'BS_270'],  # EBIT , total_assets 
    'Return_on_equity': ['IS_200', 'BS_400'],  # net_income , equity 
    'profitability_of_operating_expenses': ['IS_200','IS_040', 'IS_040'],  # net_income_from_operating , total_operating_expenses 
    'Return_on_sales': ['IS_200', 'IS_020'],  # net_income , net_sales 
    'operating_profit_margin': ['IS_200','IS_040', 'IS_020'],  # NOPAT, net_sales 
    # 'gross_profit_margin': ['IS_020', ['IS_010','IS_003','IS_004','IS_007','IS_008','IS_009']],  # gross_profit , net_sales 
}

# Create a DataFrame to store the results of the new ratios
results_5 = []

# Iterate through the pivot table to calculate the new ratios
for index, row in pivot_df_5.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in ratio_functions_5.items():
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                if input_name in ['BS_220', 'BS_200']:  
                    prev_q0_value = get_previous_year_q0_value(stock_code, year, input_name)
                    current_value = row[input_name] if input_name in row.index else None
                    if current_value is not None and prev_q0_value is not None:
                        avg_value = (current_value + prev_q0_value) / 2
                        input_values.append(avg_value)
                    else:
                        input_values.append(None)
                else:
                    input_values.append(row[input_name] if input_name in row.index else None)
        
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            results_5.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert the results to a DataFrame
ratios_df_5 = pd.DataFrame(results_5)

# Display or export the DataFrame
ratios_df_5


,stock_code,year,quarter,ratio_code,data
0,MBS,2016,0,return_on_assets,0.004733
1,MBS,2016,0,Basic_Earning_Power_Ratio,0.004780
2,MBS,2016,0,Return_on_equity,0.012576
3,MBS,2016,0,profitability_of_operating_expenses,-0.947453
4,MBS,2016,0,Return_on_sales,0.032551
...,...,...,...,...,...
1439,VND,2024,3,Basic_Earning_Power_Ratio,NaN
1440,VND,2024,3,Return_on_equity,NaN
1441,VND,2024,3,profitability_of_operating_expenses,NaN
1442,VND,2024,3,Return_on_sales,NaN


## phân tích cashflow


In [ ]:
# # Earnings Before Interest, Taxes, Depreciation and Amortization (EBITDA) = EBIT(IS_050) + Depreciation and Amortization (CF_002)
# def EBITDA(EBIT, depreciation_and_amortization):
#     return EBIT + depreciation_and_amortization
# # Cash Flow Adequacy Ratio = Cash Flow from Operations/ Capital_Expenditures+Debt_Repayments+Dividends_Paid
# # Free Cash Flow (FCF)=Operating_Net_Cash_Flow (CF_020) − Capital_Expenditures_(CAPEX) ( CF_021+CF_023) - Chi trả cổ tức (CF_036)
# def free_cash_flow(operating_net_cash_flow, capital_expenditures, dividends_paid):
#     return operating_net_cash_flow - capital_expenditures - dividends_paid

# # free cash flow to the operating cash flow ratio = free_cash_flow / operating_net_cash_flow (CF_020)
# def free_cash_flow_to_operating_cash_flow_ratio(free_cash_flow, operating_net_cash_flow):
#     return free_cash_flow / operating_net_cash_flow

# # cash debt coverage ratio = operating_net_cash_flow(CF_020)/ avg_total_liabilities
# def cash_debt_coverage_ratio(operating_net_cash_flow, avg_total_liabilities):
#     return operating_net_cash_flow / avg_total_liabilities

# #Hệ số đảm bảo khả năng chi trả lãi vay: cash interest covarage(CIC)= operating_net_cash_flow(CF_020) + interest_expense(IS_023) / interest_expense(IS_023)
# def cash_interest_coverage(operating_net_cash_flow, interest_expense):
#     return operating_net_cash_flow + interest_expense / interest_expense

# # Hệ số khả năng chi trả cổ tức : Dividend payment ratio = perating_net_cash_flow(CF_020)/ dividends_payables (TMBCTC)

# # cash return on assets = operating_net_cash_flow(CF_020) / avg_total_assets
# def cash_return_on_assets(operating_net_cash_flow, avg_total_assets):
#     return operating_net_cash_flow / avg_total_assets

# #cash return on fixed assets = operating_net_cash_flow(CF_020) / avg_fixed_assets
# def cash_return_on_fixed_assets(operating_net_cash_flow, avg_fixed_assets):
#     return operating_net_cash_flow / avg_fixed_assets

# # CFO to total equity = operating_net_cash_flow(CF_020) / avg_total_equity
# def CFO_to_total_equity(operating_net_cash_flow, avg_total_equity):
#     return operating_net_cash_flow / avg_total_equity

# # cash flow return on investment = Net_Cash_Flow_before_Tax_and_Interest_Payments(CF_050 +CF_014 + CF_015) / VỐN SỬ DỤNG BÌNH QUÂN (BS_400+BS_330)

# # Vốn cổ phần phổ thông(Common Equity) = BS_411 - BS_415
# # Cash return on shareholders = operating_net_cash_flow(CF_020)- cổ tức ưu đãi/ số cổ phiếu phổ thông bình quân trong kỳ
# # cas flow from sales to sales = operating_net_cash_flow(CF_020)- CF_036/ net_sales(IS_010)
# def cash_flow_from_sales_to_sales(operating_net_cash_flow, net_sales):
#     return operating_net_cash_flow / net_sales

# # cash flow margin = operating_net_cash_flow(CF_020) / tổng doanh thu thuần (IS_010+ IS_021)
# def cash_flow_margin(operating_net_cash_flow, total_revenue):
#     return operating_net_cash_flow / total_revenue

# # Earning quality ratio = operating_net_cash_flow(CF_020) / net_income(IS_060)
# def earning_quality_ratio(operating_net_cash_flow, net_income):
#     return operating_net_cash_flow / net_income
# # free cash flows to the firm (FCFF) = EBIT (IS_050 +IS_023). (1 - tax rate) + Depreciation  - Capital Expenditures - Change in Net Working Capital 


In [13]:
import pandas as pd

# Create a new pivot table for the cash flow ratios in pivot_df_7
pivot_df_6 = data_df.pivot_table(index=['stock_code', 'year', 'quarter'], 
                                 columns='category_code', 
                                 values='data', 
                                 aggfunc='sum')

# Function to get the value of quarter 0 from the previous year for a given stock_code
def get_previous_year_q0_value(stock_code, year, category_code):
    try:
        return pivot_df_6.loc[(stock_code, year - 1, 0), category_code]
    except KeyError:
        return None

# Ratio calculation functions
def EBITDA(EBIT, depreciation_and_amortization):
    return EBIT + depreciation_and_amortization

def free_cash_flow(operating_net_cash_flow, capital_expenditures, dividends_paid):
    return operating_net_cash_flow - capital_expenditures - dividends_paid

def free_cash_flow_to_operating_cash_flow_ratio(free_cash_flow, operating_net_cash_flow):
    return free_cash_flow / operating_net_cash_flow if operating_net_cash_flow else None

def cash_debt_coverage_ratio(operating_net_cash_flow, avg_total_liabilities):
    return operating_net_cash_flow / avg_total_liabilities if avg_total_liabilities else None

def cash_interest_coverage(operating_net_cash_flow, interest_expense):
    return (operating_net_cash_flow + interest_expense) / interest_expense if interest_expense else None

def cash_return_on_assets(operating_net_cash_flow, avg_total_assets):
    return operating_net_cash_flow / avg_total_assets if avg_total_assets else None

def cash_return_on_fixed_assets(operating_net_cash_flow, avg_fixed_assets):
    return operating_net_cash_flow / avg_fixed_assets if avg_fixed_assets else None

def CFO_to_total_equity(operating_net_cash_flow, avg_total_equity):
    return operating_net_cash_flow / avg_total_equity if avg_total_equity else None

def cash_flow_from_sales_to_sales(operating_net_cash_flow, net_sales):
    return operating_net_cash_flow / net_sales if net_sales else None

def cash_flow_margin(operating_net_cash_flow, total_revenue):
    return operating_net_cash_flow / total_revenue if total_revenue else None

def earning_quality_ratio(operating_net_cash_flow, net_income):
    return operating_net_cash_flow / net_income if net_income else None
def net_interest_margin(net_interest_income, avg_earning_assets):
    return net_interest_income / avg_earning_assets if avg_earning_assets else None
# Dictionary to map ratios to category codes (with average calculations for avg fields)
cash_flow_ratio_functions_6 = {
    # 'EBITDA': ['IS_050', 'CF_002'],  # EBIT (IS_050), depreciation_and_amortization (CF_002)
    # 'free_cash_flow': ['CF_060', 'CF_061', 'CF_038'],  # operating_net_cash_flow , capital_expenditures , dividends_paid 
    # 'free_cash_flow_to_operating_cash_flow_ratio': ['free_cash_flow', 'CF_024'],  # free_cash_flow, operating_net_cash_flow 
    'cash_debt_coverage_ratio': ['CF_060', 'BS_300'],  # operating_net_cash_flow , avg_total_liabilities 
    'cash_interest_coverage': ['CF_060', 'IS_052'],  # operating_net_cash_flow , interest_expense
    'cash_return_on_assets': ['CF_060', 'BS_270'],  # operating_net_cash_flow , avg_total_assets 
    'cash_return_on_fixed_assets': ['CF_060', 'BS_220'],  # operating_net_cash_flow , avg_fixed_assets 
    'CFO_to_total_equity': ['CF_060', 'BS_400'],  # operating_net_cash_flow , avg_total_equity 
    'cash_flow_from_sales_to_sales': ['CF_060', 'IS_020'],  # operating_net_cash_flow , net_sales 
    'cash_flow_margin': ['CF_060', 'IS_020'],  # operating_net_cash_flow , total_revenue 
    'earning_quality_ratio': ['CF_060', 'IS_200'],  # operating_net_cash_flow , net_income 
    'net_interest_margin': ['IS_001', 'BS_110'],  # net_interest_income , avg_earning_assets
}

# Create a DataFrame to store the results of the new cash flow ratios
cash_flow_results_6 = []

# Iterate through the pivot table to calculate the cash flow ratios
for index, row in pivot_df_6.iterrows():
    stock_code, year, quarter = index
    
    for ratio, inputs in cash_flow_ratio_functions_6.items():
        input_values = []
        for input_name in inputs:
            if isinstance(input_name, list):  # Sum for cases like capital_expenditures or total_revenue
                value_sum = sum([row[i] for i in input_name if i in row.index])
                input_values.append(value_sum)
            else:
                if input_name in ['BS_400', 'BS_220', 'BS_270', 'BS_300']:  
                    prev_q0_value = get_previous_year_q0_value(stock_code, year, input_name)
                    current_value = row[input_name] if input_name in row.index else None
                    if current_value is not None and prev_q0_value is not None:
                        avg_value = (current_value + prev_q0_value) / 2
                        input_values.append(avg_value)
                    else:
                        input_values.append(None)
                else:
                    input_values.append(row[input_name] if input_name in row.index else None)
        
        # Check if all required data is available
        if None not in input_values:
            # Call the corresponding function to calculate the ratio
            ratio_value = globals()[ratio](*input_values)
            cash_flow_results_6.append({
                'stock_code': stock_code,
                'year': year,
                'quarter': quarter,
                'ratio_code': ratio,
                'data': ratio_value
            })

# Convert the results to a DataFrame
ratios_df_6 = pd.DataFrame(cash_flow_results_6)

# Display or export the DataFrame
ratios_df_6


,stock_code,year,quarter,ratio_code,data
0,MBS,2016,0,cash_interest_coverage,NaN
1,MBS,2016,0,cash_flow_from_sales_to_sales,NaN
2,MBS,2016,0,cash_flow_margin,NaN
3,MBS,2016,0,earning_quality_ratio,NaN
4,MBS,2016,0,net_interest_margin,0.012932
...,...,...,...,...,...
1595,VND,2024,3,CFO_to_total_equity,NaN
1596,VND,2024,3,cash_flow_from_sales_to_sales,NaN
1597,VND,2024,3,cash_flow_margin,NaN
1598,VND,2024,3,earning_quality_ratio,NaN


In [14]:
# Concatenate all the DataFrames
ratios_df = pd.concat([ratios_df_1, ratios_df_2, ratios_df_3, ratios_df_4, ratios_df_5, ratios_df_6], ignore_index=True)

# Display the concatenated DataFrame
ratios_df

,stock_code,year,quarter,ratio_code,data
0,MBS,2016,0,equity_ratio,0.376350
1,MBS,2016,0,long_term_asset_self_financing_ratio,8.432240
2,MBS,2016,0,fixed_asset_self_financing_ratio,132.743137
3,MBS,2016,0,general_solvency_ratio,1.603464
4,MBS,2016,0,return_on_investment,0.164881
...,...,...,...,...,...
8375,VND,2024,3,CFO_to_total_equity,NaN
8376,VND,2024,3,cash_flow_from_sales_to_sales,NaN
8377,VND,2024,3,cash_flow_margin,NaN
8378,VND,2024,3,earning_quality_ratio,NaN


In [20]:
ratios_df.rename(columns={'ratio_code': 'function_name'}, inplace=True)

In [18]:
map_df = pd.read_csv(r'D:\python\financial statement prj\group_2_chatbot_financial_statement\csv\map_ratio_code.csv')
map_df.drop(columns=['ratio_name'], inplace=True)

In [19]:
map_df

,ratio_code,function_name
0,ER,equity_ratio
1,LTASFR,long_term_asset_self_financing_ratio
2,FASFR,fixed_asset_self_financing_ratio
3,GSR,general_solvency_ratio
4,ROI,return_on_investment
5,BEP,basic_earning_power
6,DTAR,debt_to_assets_ratio
7,DTER,debt_to_equity_ratio
8,STDTAR,short_term_debt_to_assets_ratio
9,ICR,interest_coverage_ratio


In [22]:
final_df = pd.merge(ratios_df, map_df, on='function_name', how='left')
final_df

,stock_code,year,quarter,function_name,data,ratio_code
0,MBS,2016,0,equity_ratio,0.376350,ER
1,MBS,2016,0,long_term_asset_self_financing_ratio,8.432240,LTASFR
2,MBS,2016,0,fixed_asset_self_financing_ratio,132.743137,FASFR
3,MBS,2016,0,general_solvency_ratio,1.603464,GSR
4,MBS,2016,0,return_on_investment,0.164881,ROI
...,...,...,...,...,...,...
8375,VND,2024,3,CFO_to_total_equity,NaN,CFOTTE
8376,VND,2024,3,cash_flow_from_sales_to_sales,NaN,CFFSTS
8377,VND,2024,3,cash_flow_margin,NaN,CFM
8378,VND,2024,3,earning_quality_ratio,NaN,EQR


In [23]:
final_df.drop(columns=['function_name'], inplace=True)

In [24]:
final_df.to_csv('financial_ratios_sec.csv', index=False)